In [15]:
import sqlite3
import json

con = sqlite3.connect("git.db")

In [14]:
cur = con.cursor()

# specify the path to the JSON file
json_file_path = "events.json"

# open the file and load its contents into a Python object
with open(json_file_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)
## Parsing
for entry in json_data:
    ## CreateEvent Parsing
    if entry['type'] == "CreateEvent":
        break
    ## PushEvent Parsing
    if entry['type'] == "PushEvent":
        break
    ## IssueCommentEvent Parsing
    if entry['type'] == "IssueCommentEvent":
        break
    ## PullRequestingEvent
    if entry['type'] == "PullRequestEvent":
        break

In [ ]:
## Searching and Examples of new uses